# Simulated Dataset Analyses

To run the following code, you need to first download the supplementary results. Please see the publication for a download link.

In [2]:
import datetime
import math
import pandas as pd
import pathlib
import pingouin as pg
from plotly import graph_objects as go
from plotly import io as pio
from plotly.subplots import make_subplots
from scipy.stats import f_oneway, bartlett
import stdpopsim

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"


RESULTS = pathlib.Path("results")
MODELS = [model.id for model in stdpopsim.get_species("HomSap").demographic_models]

C0 = "rgb(0, 150, 130)"

PLOTS_DIR = pathlib.Path("plots")
PLOTS_DIR.mkdir(exist_ok=True)

/Users/julia/micromamba/envs/pandora/lib/python3.11/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:158: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.head(1) - 1), rate=0
/Users/julia/micromamba/envs/pandora/lib/python3.11/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:161: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.tail(1) + 1), rate=0


In [3]:
def load(pth):
    if not pathlib.Path(pth).exists():
        return None
    df = pd.read_parquet(pth)
    df["idx"] = df.apply(lambda x: "_".join([
        x.model,
        str(x.seq_len),
        str(x.missing),
        str(x.noise),
    ]), axis=1)
    df.set_index("idx", inplace=True)
    df.sort_index(inplace=True)
    return df

In [4]:
mds_conv_5 = load(RESULTS / "MDS/convergence_5.parquet")
mds_conv_1 = load(RESULTS / "MDS/convergence_1.parquet")
mds_no_conv = load(RESULTS / "MDS/no_convergence.parquet")


pca_conv_5 = load(RESULTS / "PCA/convergence_5.parquet")
pca_conv_1 = load(RESULTS / "PCA/convergence_1.parquet")
pca_no_conv = load(RESULTS / "PCA/no_convergence.parquet")

# Influence of seq_len, missing, noise on the PS

In [186]:
def plot_missing(df, value="PS", run_anova=True, plot_prefix="PCA"):
    data = df.loc[lambda x: (x.seq_len == 1e8) & (x.noise == 0.0)] # & (x.missing != 0.01)]

    comp_data = [group[value].values for missing, group in data.groupby("missing")]

    if run_anova:
        b = bartlett(*comp_data)
        if b.pvalue < 0.05:
            print("-> H0 can be rejected: Variances are not equal -> Welch's ANOVA")
            # transform data into a dataframe for pg
            _data = pd.DataFrame([(missing, v) for missing, group in data.groupby("missing") for v in group[value].values], columns=["missing", value])

            # Welch ANOVA (if variances are not equal); if p-value < 0.05, means are not equal
            w = pg.welch_anova(dv=value, between="missing", data=_data)
            print("- Welch", w.F.mean().round(3), w["p-unc"].mean().round(3))
        else:
            print("-> H0 cannot be rejected -> regular ANOVA")
            # regular anova (only if variances are equal); if p-value < 0.05, means are not equal
            a = f_oneway(*comp_data)
            print("- ANOVA", round(a.statistic, 3), round(a.pvalue, 3), a.pvalue)

    fig = go.Figure()

    for missing, group in data.groupby("missing"):
        name = f"{int(missing * 100)}%"
        fig.add_trace(
            go.Box(
                name=name,
                y=group[value].values,
                boxmean=True,
                marker_color=C0
            )
        )


    fig.update_xaxes(title="Proportion of missing data")
    fig.update_yaxes(title=value)
    fig.update_layout(
        template="plotly_white",
        height=500,
        width=750,
        showlegend=False,
    )

    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{plot_prefix}_{value}_per_missing.pdf")

    return fig


In [187]:
plot_missing(pca_conv_5, "PS", plot_prefix="PCA")

-> H0 cannot be rejected -> regular ANOVA
- ANOVA 7.838 0.0 6.183941519293507e-06


In [188]:
plot_missing(mds_conv_5, "PS", False, plot_prefix="MDS")

In [189]:
def plot_seq_len(df, value="PS", plot_prefix="PCA"):
    data = df.loc[lambda x: (x.missing == 0) & (x.noise == 0)]
    data = data.groupby(by=["seq_len", "model"])[value].mean()

    fig = go.Figure()

    for seq_len, group in data.groupby(level=0):
        exponent = math.log10(seq_len)
        name = f"10<sup>{int(exponent)}</sup>"
        fig.add_trace(
            go.Box(
                name=name,
                y=group.values,
                boxmean=True,
                marker_color=C0
            )
        )

    fig.update_xaxes(title="Sequence length")
    fig.update_yaxes(title=value)
    fig.update_layout(
        template="plotly_white",
        width=750,
        height=500,
        showlegend=False,
    )
    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{plot_prefix}_{value}_per_seq_len.pdf")
    return fig

In [190]:
plot_seq_len(pca_conv_5, "PS", plot_prefix="PCA")

In [191]:
plot_seq_len(mds_conv_5, "PS", plot_prefix="MDS")

In [192]:
def plot_noise(df, value="PS", run_anova=True, plot_prefix="PCA"):
    data = df.loc[lambda x: (x.seq_len == 1e8) & (x.missing == 0.0)]

    comp_data = [group[value].values for noise, group in data.groupby("noise")]

    if run_anova:
        b = bartlett(*comp_data)
        if b.pvalue < 0.05:
            print("-> H0 can be rejected: Variances are not equal -> Welch's ANOVA")
            # transform data into a dataframe for pg
            _data = pd.DataFrame([(noise, v) for noise, group in data.groupby("noise") for v in group[value].values], columns=["noise", value])

            # Welch ANOVA (if variances are not equal); if p-value < 0.05, means are not equal
            w = pg.welch_anova(dv=value, between="noise", data=_data)
            print("- Welch", w.F.mean().round(3), w["p-unc"].mean().round(3))
        else:
            print("-> H0 cannot be rejected -> regular ANOVA")
            # regular anova (only if variances are equal); if p-value < 0.05, means are not equal
            a = f_oneway(*comp_data)
            print("- ANOVA", round(a.statistic, 3), round(a.pvalue, 3), a.pvalue)

    fig = go.Figure()

    for noise, group in data.groupby("noise"):
        name = f"{int(noise * 100)}%"
        fig.add_trace(
            go.Box(
                name=name,
                y=group[value].values,
                boxmean=True,
                marker_color=C0
            )
        )


    fig.update_xaxes(title="Proportion of random noise in data")
    fig.update_yaxes(title=value)
    fig.update_layout(
        template="plotly_white",
        height=500,
        width=750,
        showlegend=False,
    )
    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{plot_prefix}_{value}_per_noise.pdf")
    return fig


In [193]:
plot_noise(pca_conv_5, "PS", True, plot_prefix="PCA")

-> H0 cannot be rejected -> regular ANOVA
- ANOVA 50.347 0.0 7.365376893013021e-15


In [194]:
plot_noise(mds_conv_5, "PS", False, plot_prefix="MDS")

## Exemplary results for one demopgraphic model: AncientEurope_4A21

In [5]:
from pandora.embedding import from_smartpca
from pandora.plotting import plot_populations

MODEL = "AncientEurope_4A21"
base_local = pathlib.Path("results/PCA") / MODEL

seq_lens = ["1e5", "1e6", "1e7", "1e8"]
missing_values = [0.0, 0.01, 0.05, 0.1, 0.2, 0.5]
noise_values = [0.0, 0.1, 0.2, 0.5]

In [6]:
# Plot PCA of dataset (and PS)

def get_ps(seq_len, missing, noise):
    ps = pca_conv_5.loc[lambda x: (x.model == MODEL) & (x.missing == missing) & (x.seq_len == eval(seq_len)) & (x.noise == noise)].PS.mean()
    return round(ps, 2)


def _fmt_seq_len(seq_len):
    _, exponent = seq_len.split("e")
    return f"10<sup>{exponent}</sup>"



def plot_pca_per_seq_len():
    missing = 0.0
    noise = 0.0

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"{_fmt_seq_len(seq_len)} (PS={get_ps(seq_len, missing, noise)})" for seq_len in seq_lens]
    )

    for i, seq_len in enumerate(seq_lens):
        eval_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.eval"
        evec_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.evec"

        pca = from_smartpca(evec_file, eval_file)
        pca.embedding["population"] = pca.embedding["population"].apply(lambda x: x.replace("_", " "))
        _fig = plot_populations(pca)

        [fig.add_trace(d.update(dict(showlegend=i == 0)), row=i // 2 + 1, col=i % 2 + 1) for d in _fig.data]

        fig.update_xaxes(title=f"PC 1 ({round(pca.explained_variances[0] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)
        fig.update_yaxes(title=f"PC 2 ({round(pca.explained_variances[1] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)


    fig.update_layout(
        template="plotly_white",
        height=1000,
        width=850,
        title=MODEL,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.1,
            xanchor="right",
            x=0.9
        )
    )

    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{MODEL}_seq_len.pdf")

    return fig


def plot_pca_per_missing():
    seq_len = "1e8"
    noise = 0.0

    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[f"{missing * 100}% (PS={get_ps(seq_len, missing, noise)})" for missing in missing_values]
    )

    _flips = {  # This does not change results, we simply want to have all embeddings to have the same orientation for easier comparison
            0.0: [0],
            0.01: [0],
            0.05: [],
            0.1: [],
            0.2: [1],
            0.5: [1]
        }


    for i, missing in enumerate(missing_values):
        eval_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.eval"
        evec_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.evec"

        pca = from_smartpca(evec_file, eval_file)
        pca.embedding["population"] = pca.embedding["population"].apply(lambda x: x.replace("_", " "))

        for flip_dim in _flips[missing]:
            pca.embedding[f"D{flip_dim}"] = -pca.embedding[f"D{flip_dim}"]
            pca.embedding_matrix[:, flip_dim] = -pca.embedding_matrix[:, flip_dim]

        _fig = plot_populations(pca)

        [fig.add_trace(d.update(dict(showlegend=i == 0)), row=i // 2 + 1, col=i % 2 + 1) for d in _fig.data]

        fig.update_xaxes(title=f"PC 1 ({round(pca.explained_variances[0] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)
        fig.update_yaxes(title=f"PC 2 ({round(pca.explained_variances[1] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)


    fig.update_layout(
        template="plotly_white",
        height=1000,
        width=850,
        title=MODEL,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.1,
            xanchor="right",
            x=0.9
        )
    )



    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{MODEL}_missing.pdf")

    return fig


def plot_pca_per_noise():
    seq_len = "1e8"
    missing = 0.0

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"{noise * 100}% (PS={get_ps(seq_len, missing, noise)})" for noise in noise_values]
    )

    _flips = {  # This does not change results, we simply want to have all embeddings to have the same orientation for easier comparison
            0.0: [0],
            0.1: [],
            0.2: [1],
            0.5: [0, 1]
        }


    for i, noise in enumerate(noise_values):
        eval_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.eval"
        evec_file = base_local / f"{MODEL}_{seq_len}_500_missing_{missing}_noise_{noise}.evec"

        pca = from_smartpca(evec_file, eval_file)
        pca.embedding["population"] = pca.embedding["population"].apply(lambda x: x.replace("_", " "))

        for flip_dim in _flips[noise]:
            pca.embedding[f"D{flip_dim}"] = -pca.embedding[f"D{flip_dim}"]
            pca.embedding_matrix[:, flip_dim] = -pca.embedding_matrix[:, flip_dim]

        _fig = plot_populations(pca)

        [fig.add_trace(d.update(dict(showlegend=i == 0)), row=i // 2 + 1, col=i % 2 + 1) for d in _fig.data]

        fig.update_xaxes(title=f"PC 1 ({round(pca.explained_variances[0] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)
        fig.update_yaxes(title=f"PC 2 ({round(pca.explained_variances[1] * 100, 1)}%)", row=i // 2 + 1, col=i % 2 + 1)


    fig.update_layout(
        template="plotly_white",
        height=700,
        width=850,
        title=MODEL,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.1,
            xanchor="right",
            x=0.9
        )
    )



    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{MODEL}_noise.pdf")

    return fig

In [197]:
_ = plot_pca_per_missing()

In [198]:
_ = plot_pca_per_seq_len()

In [199]:
plot_pca_per_noise()

# Bootstrap Convergence Criterion

- comparison of number of replicates computed
- runtime comparison 5% vs 1% vs disabled convergence detection
- influence on PS and PSV values

In [7]:
psv_mds_conv_5 = pd.read_parquet(RESULTS / "MDS/PSVs_convergence_5.parquet")
psv_mds_conv_1 = pd.read_parquet(RESULTS / "MDS/PSVs_convergence_1.parquet")
psv_mds_no_conv = pd.read_parquet(RESULTS / "MDS/PSVs_no_convergence.parquet")

psv_pca_conv_5 = pd.read_parquet(RESULTS / "PCA/PSVs_convergence_5.parquet")
psv_pca_conv_1 = pd.read_parquet(RESULTS / "PCA/PSVs_convergence_1.parquet")
psv_pca_no_conv = pd.read_parquet(RESULTS / "PCA/PSVs_no_convergence.parquet")

In [267]:
def _get_data(mode):
    if mode == "PCA":
        c5 = pca_conv_5.loc[lambda x: (x.seq_len == 1e8) & (x.missing == 0) & (x.noise == 0)]
        c1 = pca_conv_1
        nc = pca_no_conv
    else:
        c5 = mds_conv_5.loc[lambda x: (x.seq_len == 1e8) & (x.missing == 0) & (x.noise == 0)]
        c1 = mds_conv_1
        nc = mds_no_conv

    assert (c5.index == c1.index).all()
    assert (c5.index == nc.index).all()
    return c5, c1, nc

def metrics(mode="PCA"):
    c5, c1, nc = _get_data(mode)

    c5_s = nc.runtime / c5.runtime
    c1_s = nc.runtime / c1.runtime
    print(f"----------- {mode} -------------")
    print(f"Speedup: \n"
          f"> 5% vs nc: {round(c5_s.mean(), 2)} ± {round(c5_s.std(), 2)} ({round(c5.n_replicates.mean())} replicates)\n"
          f"> 1% vs nc: {round(c1_s.mean(), 2)} ± {round(c1_s.std(), 2)} ({round(c1.n_replicates.mean())} replicates)"
    )

    print()

    c5_abs = (c5.PS - nc.PS).abs()
    c1_abs = (c1.PS - nc.PS).abs()

    print(f"PS Absolute Deviation: \n"
          f"> 5% vs nc: {round(c5_abs.mean(), 2)} ± {round(c5_abs.std(), 2)}\n"
          f"> 1% vs nc: {round(c1_abs.mean(), 2)} ± {round(c1_abs.std(), 2)}")

    print()

    c5_ps = (100 * (c5_abs) / nc.PS).abs()
    c1_ps = (100 * (c1_abs) / nc.PS).abs()

    print(f"PS Relative Deviation: \n"
          f"> 5% vs nc: {round(c5_ps.mean(), 2)}% ± {round(c5_ps.std(), 2)}%\n"
          f"> 1% vs nc: {round(c1_ps.mean(), 2)}% ± {round(c1_ps.std(), 2)}%")


def plot_ps_deviation(mode="PCA"):
    c5, c1, nc = _get_data(mode)

    ps_c5 = c5.PS - nc.PS
    ps_c1 = c1.PS - nc.PS

    fig = go.Figure(
        [
            go.Box(
                name="5%",
                y=ps_c5.abs().values,
                marker_color=C0,
                boxmean=True,
            ),
            go.Box(
                name="1%",
                y=ps_c1.abs().values,
                marker_color=C0,
                boxmean=True,
            )
        ]
    )
    fig.update_xaxes(title="Convergence Tolerance")
    fig.update_yaxes(title="PS Deviation")
    fig.update_layout(showlegend=False, title=f"PS Deviation ({mode} analyses)")

    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{mode}_ps_deviation.pdf")

    return fig


def plot_speedup(mode="PCA"):
    c5, c1, nc = _get_data(mode)

    s_c5 = nc.runtime / c5.runtime
    s_c1 = nc.runtime / c1.runtime

    fig = go.Figure(
        [
            go.Box(
                name="5%",
                y=s_c5.values,
                marker_color=C0,
                boxmean=True
            ),
            go.Box(
                name="1%",
                y=s_c1.values,
                marker_color=C0,
                boxmean=True
            )
        ]
    )
    fig.update_xaxes(title="Convergence Tolerance")
    fig.update_yaxes(title="Speedup", ticksuffix="x")
    fig.update_layout(showlegend=False, title=f"Speedup ({mode} analyses)")

    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{mode}_speedup.pdf")

    return fig


def plot_psv_deviation(mode="PCA"):
    if mode == "PCA":
        c5 = psv_pca_conv_5
        c1 = psv_pca_conv_1
        nc = psv_pca_no_conv
    else:
        c5 = psv_mds_conv_5
        c1 = psv_mds_conv_1
        nc = psv_mds_no_conv

    deviations_c5 = []
    deviations_c1 = []

    for model in MODELS:
        model_c5 = c5.loc[lambda x: x.model == model]
        model_c1 = c1.loc[lambda x: x.model == model]
        model_nc = nc.loc[lambda x: x.model == model]

        assert (model_c5.index == model_nc.index).all()
        assert (model_c1.index == model_nc.index).all()

        dev_c5 = model_c5.PSV - model_nc.PSV
        dev_c1 = model_c1.PSV - model_nc.PSV

        deviations_c5.extend(dev_c5.to_list())
        deviations_c1.extend(dev_c1.to_list())

    deviations_c5 = pd.Series(deviations_c5)
    deviations_c1 = pd.Series(deviations_c1)

    fig  = go.Figure(
        [
            go.Box(
                name="5%",
                y=deviations_c5.abs().values,
                marker_color=C0,
                boxmean=True
            ),
            go.Box(
                name="1%",
                y=deviations_c1.abs().values,
                marker_color=C0,
                boxmean=True
            )
        ]
    )

    fig.update_xaxes(title="Convergence Tolerance")
    fig.update_yaxes(title="PSV Deviation")
    fig.update_layout(showlegend=False, title=f"PSV Deviation ({mode} analyses)")

    fig.write_image("/tmp/foo.pdf")
    fig.write_image(PLOTS_DIR / f"{mode}_psv_deviation.pdf")

    return fig

In [268]:
metrics("PCA")
print()
metrics("MDS")

----------- PCA -------------
Speedup: 
> 5% vs nc: 2.57 ± 0.57 (20 replicates)
> 1% vs nc: 1.35 ± 0.38 (63 replicates)

PS Absolute Deviation: 
> 5% vs nc: 0.01 ± 0.01
> 1% vs nc: 0.0 ± 0.0

PS Relative Deviation: 
> 5% vs nc: 0.7% ± 0.58%
> 1% vs nc: 0.34% ± 0.53%

----------- MDS -------------
Speedup: 
> 5% vs nc: 2.17 ± 0.64 (20 replicates)
> 1% vs nc: 1.7 ± 0.72 (34 replicates)

PS Absolute Deviation: 
> 5% vs nc: 0.0 ± 0.01
> 1% vs nc: 0.0 ± 0.0

PS Relative Deviation: 
> 5% vs nc: 0.3% ± 0.72%
> 1% vs nc: 0.23% ± 0.66%


In [269]:
plot_speedup("PCA")
plot_speedup("MDS")

In [290]:
plot_ps_deviation("PCA")
plot_ps_deviation("MDS")

In [291]:
plot_psv_deviation("PCA")
plot_psv_deviation("MDS")